<a href="https://colab.research.google.com/github/XingxinHE/Python-for-Data-Analysis-Notes/blob/master/7_2_Data_Transformation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [103]:
import pandas as pd
import numpy as np

## Removing Duplicates

In [104]:
data = pd.DataFrame({'k1': ['one', 'two'] * 3 + ['two'],
          'k2': [1, 1, 2, 3, 3, 4, 4]})
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


In [105]:
data.duplicated() #这个是看每一row有没有duplicate的，
#符合条件是以row为基准，例如最后那一row，只和row 5一样

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

In [106]:
data.drop_duplicates() #把duplicate的row给drop掉

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


以上这些方法都是默认考虑所有columns的，当然你可以specify which column

In [107]:
data['v1'] = range(7)
data

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
5,two,4,5
6,two,4,6


In [108]:
data.drop_duplicates(['k1']) #如果只考虑k1的话，那么从第三行开始就被删除了

,k1,k2,v1
0,one,1,0
1,two,1,1


In [109]:
data.drop_duplicates(['k1', 'k2'], keep='last')
#当specify两个column的时候，要满足两个column都是duplicate才可以drop
#keep=last是从下开始检索，保留最后面的，删除前面和它重复的

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
6,two,4,6


## Transforming Data Using a Function or Mapping
这个专题很重要，常用到

In [110]:
data = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon',
           'Pastrami', 'corned beef', 'Bacon',
           'pastrami', 'honey ham', 'nova lox'],
           'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


我们需要把每种食物的名字转换成肉类，比如salami是猪肉

In [111]:
meat_to_animal = {
    'bacon': 'pig',
    'pulled pork': 'pig',
    'pastrami': 'cow',
    'corned beef': 'cow',
    'honey ham': 'pig',
    'nova lox': 'salmon'
}

这种经典的map function需要的是一个dict

当这里有个问题，前面df某些value的大小写不一致，如bacon和Bacon，这在data cleaning里面经常碰到，需要做以下操作

In [112]:
lowercased = data['food'].str.lower() #将food column转换成小写格式
lowercased

0          bacon
1    pulled pork
2          bacon
3       pastrami
4    corned beef
5          bacon
6       pastrami
7      honey ham
8       nova lox
Name: food, dtype: object

In [113]:
data['animal'] = lowercased.map(meat_to_animal) #利用map function，把肉类map出来
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


我们可以把上面的几个步骤汇聚成一个，很重要！很关键！

In [114]:
data['food'].map(lambda x: meat_to_animal[x.lower()])
#解读上面这段代码：
#对data['food']进行map function， map(function)
#这个function是什么？
#是一个匿名函数lambda x
#lambda x 是把 x.lower()小写的x，也就是data['food']的value 写入meat_to_animal这个dict
#因为写入的是key,所以return的是meat_to_animal的value，即肉类

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

🌟**Using map is a convenient way to perform element-wise transformations and other data cleaning-related operations!!!!!!!!**

## Replacing Values
.replace()

In [115]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

In [116]:
data.replace(-999, np.nan) #经常会用到replace，例如这个series的-999明显是outlier，所以要replace掉

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

In [117]:
data.replace([-999, -1000], np.nan) #replace多个value

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

In [118]:
data.replace([-999, -1000], [np.nan, 0]) #2对2的replace，前提是被replace和replace物的shape一致

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

In [119]:
data.replace({-999: np.nan, -1000: 0}) #这种用dict的方法最稳妥

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

## Renaming Axis Indexes

In [120]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
          index=['Ohio', 'Colorado', 'New York'],
          columns=['one', 'two', 'three', 'four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


一种方法是设定一个function，然后map这个function来修改名字

In [121]:
transform = lambda x: x[:4].upper() #取前4个字节，然后大写它们
data.index.map(transform)

Index(['OHIO', 'COLO', 'NEW '], dtype='object')

In [122]:
data.index = data.index.map(transform)
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


上面的方法比较累赘，如果只是modify标题的大小写的话，因为pandas为此设计了更方便的function

.rename()

In [123]:
data.rename(index=str.title, columns=str.upper) #这真的强，一行代码解决了row和column的rename

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colo,4,5,6,7
New,8,9,10,11


In [124]:
#同理，rename也可以接受一个dict
data.rename(index={'OHIO': 'INDIANA'},
      columns={'three': 'peekaboo'})

,one,two,peekaboo,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


In [125]:
data.rename(index={'OHIO': 'INDIANA'}, inplace=True)
#inplace就是直接修改原df

In [126]:
data

,one,two,three,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


## Discretization and Binning

Continuous data is often discretized or otherwise separated into "bins" for analysis. Suppose you have data about a group of people in a study, and you want to group them into discrete age buckets

连续的数据通常被离散化或以其他方式分离成 "bin"进行分析。假设在一项研究中，你有一群人的数据，你想把他们分成离散的年龄段。

说大白话点，就是把一个continuous的data分成段(bin)

pd.cut()

In [127]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32] #离散化的数字
bins = [18, 25, 35, 60, 100] #bin的domain

In [128]:
cuts = pd.cut(ages, bins)
cuts #可以看到各数值分布在哪些domain之间

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

output是一个特殊的categories的object type

In [129]:
cuts.codes #看看具体在哪个分区，比如第一个(18,25]是第一个分区，所以为0

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [130]:
cuts.categories #这个则是还原在哪个分区

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]],
              closed='right',
              dtype='interval[int64]')

In [131]:
pd.value_counts(cuts) #这个可以把codes&categories统合在一起，看各分区都有多少个

(18, 25]     5
(35, 60]     3
(25, 35]     3
(60, 100]    1
dtype: int64

( means open,  [ means inclusive，这个可以change的

In [132]:
pd.cut(ages, [18, 26, 36, 61, 100], right=False)

[[18, 26), [18, 26), [18, 26), [26, 36), [18, 26), ..., [26, 36), [61, 100), [36, 61), [36, 61), [26, 36)]
Length: 12
Categories (4, interval[int64]): [[18, 26) < [26, 36) < [36, 61) < [61, 100)]

接下来就牛逼了，我们把年龄段分出来之后，肯定会把这些年龄段命名嘛，如下

In [133]:
group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']
pd.cut(ages, bins, labels=group_names)

[Youth, Youth, Youth, YoungAdult, Youth, ..., YoungAdult, Senior, MiddleAged, MiddleAged, YoungAdult]
Length: 12
Categories (4, object): [Youth < YoungAdult < MiddleAged < Senior]

前面提到的是pd.cut(值，划分的区间）

还有一种方法是pd.cut（值，多少个区间），这种方法类似于grasshopper的divide domain

In [134]:
data = np.random.rand(20)
data

array([0.4041688 , 0.8677797 , 0.41721811, 0.39351415, 0.68435493,
       0.20572293, 0.44385216, 0.30930415, 0.10666456, 0.52895713,
       0.08548425, 0.52750325, 0.47911156, 0.16833624, 0.03926753,
       0.70458647, 0.99241276, 0.66019894, 0.427483  , 0.78697545])

In [135]:
pd.cut(data, 4, precision=2) #把这些值统计出来bounds是多少（GH的思想），然后把这个bounds得出来的domain / 4
#precision是精确到最后几位数

[(0.28, 0.52], (0.75, 0.99], (0.28, 0.52], (0.28, 0.52], (0.52, 0.75], ..., (0.52, 0.75], (0.75, 0.99], (0.52, 0.75], (0.28, 0.52], (0.75, 0.99]]
Length: 20
Categories (4, interval[float64]): [(0.038, 0.28] < (0.28, 0.52] < (0.52, 0.75] < (0.75, 0.99]]

上文提及的是divide domain的方法， 下面则是根据输入value的数目进行等分，类似于GH的partition list

pd.qcut()                       
q是quantiles的意思




In [136]:
data = np.random.randn(1000) # Normally distributed
cuts = pd.qcut(data, 4) # Cut into quartiles

In [137]:
cuts

[(0.64, 3.468], (-0.676, -0.00841], (-0.00841, 0.64], (0.64, 3.468], (0.64, 3.468], ..., (-3.203, -0.676], (-0.676, -0.00841], (-0.676, -0.00841], (-0.676, -0.00841], (0.64, 3.468]]
Length: 1000
Categories (4, interval[float64]): [(-3.203, -0.676] < (-0.676, -0.00841] < (-0.00841, 0.64] <
                                    (0.64, 3.468]]

In [138]:
pd.value_counts(cuts)
#可以看到，它自动取值了4个区间，让每个区间都是250个item

(0.64, 3.468]         250
(-0.00841, 0.64]      250
(-0.676, -0.00841]    250
(-3.203, -0.676]      250
dtype: int64

## Detecting and Filtering Outliers

In [139]:
data = pd.DataFrame(np.random.randn(1000, 4))
data

,0,1,2,3
0,0.553922,-0.324934,-0.246187,0.295968
1,0.046929,0.194854,2.660654,1.224805
2,-0.051673,-1.180979,-1.501382,1.948565
3,-0.086015,0.934912,-1.336459,0.596660
4,-0.553928,-0.893952,0.081385,0.474241
...,...,...,...,...
995,0.559883,0.679119,0.366813,0.380339
996,-1.311913,0.745175,-0.031379,0.832892
997,0.360498,0.052137,-0.286622,0.125795
998,2.117886,0.085074,-0.589833,-1.665524


In [140]:
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.007005,-0.007442,0.010359,-0.015046
std,1.019108,1.011219,0.992525,1.003488
min,-3.652487,-3.533534,-2.768164,-3.082351
25%,-0.690390,-0.713112,-0.661694,-0.741177
50%,0.001342,-0.029059,-0.023450,-0.019137
75%,0.676741,0.659396,0.702434,0.665081
max,3.592117,3.542019,3.417529,2.897960


In [141]:
col = data[2] #取column 2
col

0     -0.246187
1      2.660654
2     -1.501382
3     -1.336459
4      0.081385
         ...   
995    0.366813
996   -0.031379
997   -0.286622
998   -0.589833
999    0.990189
Name: 2, Length: 1000, dtype: float64

In [142]:
col[np.abs(col) > 3] #取绝对值大于3的值

917    3.417529
Name: 2, dtype: float64

In [143]:
#上面是限制在某个column上，那么我想知道只要包含绝对值大于3的那一个row呢？
data[ (np.abs(data) > 3).any(1)]

,0,1,2,3
95,-1.740508,-3.533534,-0.827538,0.097604
217,3.592117,0.058409,0.095350,-0.510223
264,3.408458,-0.760599,-0.423252,0.084207
301,0.296822,3.542019,-0.219752,0.918776
351,-0.476796,-1.193879,-0.223629,-3.082351
409,-1.245705,3.052465,-1.566483,-0.183700
501,-0.851300,3.379888,1.220188,-0.539650
543,-3.652487,-0.423699,0.454924,-0.925265
714,3.402014,0.301703,-0.695013,2.067481
917,2.298150,2.050282,3.417529,0.596400


上面找到了outlier，那么如何对outlier进行操作呢？

有一种方法叫做caps,就是把这个极值限制在某个值域

In [144]:
data[np.abs(data)>3] = np.sign(data) * 3
data.describe() #可以看到无论是min还是max，都封装在了3

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.006255,-0.007883,0.009942,-0.014964
std,1.012493,1.006367,0.991177,1.003239
min,-3.000000,-3.000000,-2.768164,-3.000000
25%,-0.690390,-0.713112,-0.661694,-0.741177
50%,0.001342,-0.029059,-0.023450,-0.019137
75%,0.676741,0.659396,0.702434,0.665081
max,3.000000,3.000000,3.000000,2.897960


那这个np.sign()是什么函数呢？

In [145]:
np.sign(data).head()

,0,1,2,3
0,1.0,-1.0,-1.0,1.0
1,1.0,1.0,1.0,1.0
2,-1.0,-1.0,-1.0,1.0
3,-1.0,1.0,-1.0,1.0
4,-1.0,-1.0,1.0,1.0


可以看到np.sign()对那些outlier,根据正负值生成了-1和1

上面的sign * 3就是为了让它们生成-3和3

## Permutation and Random Sampling

permutation是排列组合的意思

In [146]:
df = pd.DataFrame(np.arange(5 * 4).reshape((5, 4)))
sampler = np.random.permutation(5) #创建任意顺序的5个长度(0-4)的list

In [147]:
df

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [148]:
sampler

array([2, 1, 0, 4, 3])

In [149]:
df.take(sampler) #take是给定index，它就take哪些row出来

,0,1,2,3
2,8,9,10,11
1,4,5,6,7
0,0,1,2,3
4,16,17,18,19
3,12,13,14,15


In [150]:
df.sample(n=3) #ML经典操作，随机采样3个

,0,1,2,3
0,0,1,2,3
3,12,13,14,15
2,8,9,10,11


随机采样还可以有重复

In [151]:
choices = pd.Series([5, 7, -1, 6, 4])
draws = choices.sample(n=10, replace=True) #replace = True可以有重复

In [152]:
draws

2   -1
2   -1
3    6
4    4
3    6
1    7
2   -1
4    4
0    5
3    6
dtype: int64

## Computing Indicator / Dummy Variables

If a column in a DataFrame has k distinct values, you would derive a matrix or DataFrame with k columns containing all 1s and 0s.

如果DataFrame中的一列有k个不同的值，你就会推导出一个包含k个1和0的列的矩阵或DataFrame。


.get_dummies()

In [153]:
df = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
          'data1': range(6)})
df

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [154]:
pd.get_dummies(df['key']) #对key column进行get dummies操作
#可以看到a,b,c分别被进行了One-hot coding

,a,b,c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In [155]:
dummies = pd.get_dummies(df['key'], prefix='key') #这个prefix是前缀的意思，把one-hot coding的类别都加个前缀
df_with_dummy = df[['data1']].join(dummies)
df_with_dummy

,data1,key_a,key_b,key_c
0,0,0,1,0
1,1,0,1,0
2,2,1,0,0
3,3,0,0,1
4,4,1,0,0
5,5,0,1,0


#### One-hot coding 案例！🌟

加载一个dataset

In [163]:
from google.colab import drive
drive.mount('/content/gdrive')
import os
os.chdir("/content/gdrive/My Drive/Colab Notebooks/data")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [164]:
ls

movies.dat


In [165]:
movie_attribute = ['movie_id', 'title', 'genres']
movies = pd.read_table('movies.dat', sep='::', header=None, names=movie_attribute)
movies[:10]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  This is separate from the ipykernel package so we can avoid doing imports until


,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


现在想做的是，把genres那一行，不是很多分类嘛，进行get dummies

In [169]:
all_genres = []

for x in movies.genres: #例如第一行，x就是Animation|Children's|Comedy
  all_genres.extend(x.split('|')) #把上面一长串的str以 '|' 来分割开来

genres = pd.unique(all_genres) #unique

In [170]:
genres

array(['Animation', "Children's", 'Comedy', 'Adventure', 'Fantasy',
       'Romance', 'Drama', 'Action', 'Crime', 'Thriller', 'Horror',
       'Sci-Fi', 'Documentary', 'War', 'Musical', 'Mystery', 'Film-Noir',
       'Western'], dtype=object)

In [172]:
zero_matrix = np.zeros( (len(movies), len(genres)) ) #这种方法很通用，那就是做一个zeros的 matrix，
#第一个多少行，行数是movies那个DF的行数， 多少columns？ 是前面那个类别column的总和

In [173]:
zero_matrix.shape

(3883, 18)

In [175]:
movies.shape

(3883, 3)

In [176]:
dummies = pd.DataFrame(zero_matrix, columns=genres) #因为创建的时候只是ndarray，把它变成DF，然后columns的名字也都换成电影的类别
dummies.head()

,Animation,Children's,Comedy,Adventure,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Sci-Fi,Documentary,War,Musical,Mystery,Film-Noir,Western
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


##### 先解释一波怎么运作

In [177]:
#以第一个row电影为例子
gen = movies.genres[0]
gen.split('|')
#可以得到第一row的电影的分类是下面三类

['Animation', "Children's", 'Comedy']

下面的思路便是，

在dummies这个dataframe里面，把第一row(对应movies的第一row）的Animation，Children，Comedy这三个column的值变为1

In [178]:
#通过get_indexer可以找到animation，children和comedy的column分别是第0，1，2 column
dummies.columns.get_indexer(gen.split('|'))
#只要把dummies.iloc[第一row，[哪几个column]] = 1即可

array([0, 1, 2])

#### 正式操作

In [180]:
#这里用到enumerate这个function，能得到index+value
for index, gen in enumerate(movies.genres):
  column_index = dummies.columns.get_indexer(gen.split('|'))
  dummies.iloc[index, column_index] = 1

dummies.head()
#可以看到第一个电影属于动作，儿童，喜剧，分别有1

,Animation,Children's,Comedy,Adventure,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Sci-Fi,Documentary,War,Musical,Mystery,Film-Noir,Western
0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [181]:
#接下来是把它们合起来
movies_one_hot_code_genres = movies.join(dummies.add_prefix('Genre_')) #把moview和加上前缀的dummies结合起来

#看看第一行
movies_one_hot_code_genres.iloc[0]

movie_id                                       1
title                           Toy Story (1995)
genres               Animation|Children's|Comedy
Genre_Animation                                1
Genre_Children's                               1
Genre_Comedy                                   1
Genre_Adventure                                0
Genre_Fantasy                                  0
Genre_Romance                                  0
Genre_Drama                                    0
Genre_Action                                   0
Genre_Crime                                    0
Genre_Thriller                                 0
Genre_Horror                                   0
Genre_Sci-Fi                                   0
Genre_Documentary                              0
Genre_War                                      0
Genre_Musical                                  0
Genre_Mystery                                  0
Genre_Film-Noir                                0
Genre_Western       

In [182]:
movies_one_hot_code_genres.head(1)

,movie_id,title,genres,Genre_Animation,Genre_Children's,Genre_Comedy,Genre_Adventure,Genre_Fantasy,Genre_Romance,Genre_Drama,Genre_Action,Genre_Crime,Genre_Thriller,Genre_Horror,Genre_Sci-Fi,Genre_Documentary,Genre_War,Genre_Musical,Genre_Mystery,Genre_Film-Noir,Genre_Western
0,1,Toy Story (1995),Animation|Children's|Comedy,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### 以上是categorical data，那对于numerical呢？

可以作分区，用pd.cut()

In [183]:
np.random.seed(12345)
values = np.random.rand(10)
values

array([0.92961609, 0.31637555, 0.18391881, 0.20456028, 0.56772503,
       0.5955447 , 0.96451452, 0.6531771 , 0.74890664, 0.65356987])

In [184]:
bins = [0, 0.2, 0.4, 0.6, 0.8, 1]

In [185]:
pd.get_dummies(pd.cut(values, bins))

,"(0.0, 0.2]","(0.2, 0.4]","(0.4, 0.6]","(0.6, 0.8]","(0.8, 1.0]"
0,0,0,0,0,1
1,0,1,0,0,0
2,1,0,0,0,0
3,0,1,0,0,0
4,0,0,1,0,0
5,0,0,1,0,0
6,0,0,0,0,1
7,0,0,0,1,0
8,0,0,0,1,0
9,0,0,0,1,0
